# Train with Tensorflow
This notebook shows the Tensorflow training loop.

<!--<badge>--><a href="https://colab.research.google.com/github/visual-layer/vl-datasets/blob/build-wheel/notebooks/train-tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a><!--</badge>-->

In [2]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator


2023-04-11 14:59:57.057925: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-11 14:59:57.059233: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 14:59:57.084652: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 14:59:57.085134: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 14:59:57.568240: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
# Define the dataset paths
train_path = "data/train_set/"
valid_path = "data/valid_set/"

In [4]:
# Define the preprocessing generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale=1./255)

# Define the datasets using the flow_from_directory method
train_dataset = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical')
valid_dataset = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=64,
    class_mode='categorical')

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [5]:
# Define the model architecture
model = tf.keras.applications.ResNet50(include_top=False, input_shape=(224, 224, 3))
x = layers.GlobalAveragePooling2D()(model.output)
x = layers.Dense(512, activation='relu')(x)
output = layers.Dense(len(train_dataset.class_indices), activation='softmax')(x)
model = tf.keras.models.Model(inputs=model.inputs, outputs=output)


2023-04-11 15:00:29.693110: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-11 15:00:29.693423: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


94765736/94765736 [==============================] - 9s 0us/step


In [6]:
# Define the loss function and optimizer
loss = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

In [7]:
# Compile the model
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
# Train the model
num_epochs = 10
history = model.fit(train_dataset, epochs=num_epochs, validation_data=valid_dataset)

In [ ]:
# Evaluate the model
loss, accuracy = model.evaluate(valid_dataset)
print(f"Validation Loss: {loss}, Validation Accuracy: {accuracy}")